# Pré-processamento do DF dos RREOs do parser

In [33]:
from pathlib import Path
import pandas as pd
import os

In [34]:
# Carregando o arquivo CSV
path_csv = Path("~/Projetos/Curica/curica_streamlit_censo/data/raw/financiamento/df_rreo_parser.csv")

df_financiamento_raw = pd.read_csv(path_csv, sep=";", encoding="utf-8", low_memory=False)


In [35]:
df_financiamento_raw.shape

(91, 23)

In [36]:
df_panorama_financiamento = df_financiamento_raw.copy()


## Corrigindo o nome dos Municipios

In [37]:
# Dicionário de nomes padronizados
CODIGO_IBGE_NOMES = {
    "1200013": "Acrelândia",
    "1200054": "Assis Brasil",
    "1200104": "Brasiléia",
    "1200138": "Bujari",
    "1200179": "Capixaba",
    "1200203": "Cruzeiro do Sul",
    "1200252": "Epitaciolândia",
    "1200302": "Feijó",
    "1200328": "Jordão",
    "1200336": "Mâncio Lima",
    "1200344": "Manoel Urbano",
    "1200351": "Marechal Thaumaturgo",
    "1200385": "Plácido de Castro",
    "1200393": "Porto Acre",
    "1200807": "Porto Walter",
    "1200333": "Rio Branco",
    "1200427": "Rodrigues Alves",
    "1200435": "Santa Rosa do Purus",
    "1200500": "Sena Madureira",
    "1200450": "Senador Guiomard",
    "1200690": "Tarauacá",
    "1200708": "Xapuri",
    "1200": "Acre"
}

# Corrigindo coluna 'cod_ibge': convertendo para string sem casas decimais
df_panorama_financiamento['cod_ibge'] = df_panorama_financiamento['cod_ibge'].astype('Int64').astype(str)

# Atualiza os valores da coluna 'nome' com base no dicionário
df_panorama_financiamento['nome'] = df_panorama_financiamento['cod_ibge'].map(CODIGO_IBGE_NOMES)

# Verifica se há valores não mapeados
n_municipios_nulos = df_panorama_financiamento['nome'].isna().sum()
print(f"⚠️ {n_municipios_nulos} valores de 'cod_ibge' não foram encontrados no dicionário.")

# Visualização de conferência
df_panorama_financiamento[['cod_ibge', 'nome']].drop_duplicates().sort_values('cod_ibge').head(30)


⚠️ 0 valores de 'cod_ibge' não foram encontrados no dicionário.


,cod_ibge,nome
0,1200,Acre
4,1200013,Acrelândia
8,1200054,Assis Brasil
12,1200104,Brasiléia
16,1200138,Bujari
20,1200179,Capixaba
24,1200203,Cruzeiro do Sul
28,1200252,Epitaciolândia
32,1200302,Feijó
36,1200328,Jordão


## Tratamento de missings

In [38]:
# Substitui todos os valores ausentes (NaN) por 'N/A'
df_panorama_financiamento.fillna('N/A', inplace=True)

df_panorama_financiamento.head(5)

,nome,cod_ibge,ano,valor_receita_impostos,valor_repasse_fundeb,valor_minimo_mde,valor_receita_vaaf,valor_receita_vaat,valor_receita_vaar,valor_receita_total_fundeb,...,valor_receita_salario_educacao,valor_receita_pdde,valor_receita_pnae,valor_receita_pnate,valor_receita_outras_fnde,valor_receita_convenios,valor_receita_royalties,valor_receita_operacao_credito,valor_receita_outras_outras,arquivo
0,Acre,1200,2021,"6205454230,86","1148021827,32","10164298,35",0,0,N/A,"838181940,66",...,"8908453,52",230100,"13866212,8","2140819,51","1727377,07","388182,03","25262673,35","415570,13",4421500,RREO_Estadual_12_6_2021_extraido.txt
1,Acre,1200,2022,"7502282795,52","1393948222,07","7651815,7",0,0,N/A,"1085913826,95",...,"10243068,77",212280,0,0,0,0,"51912871,88",0,0,RREO_Estadual_12_6_2022_extraido.txt
2,Acre,1200,2023,"7913861864,58","1449998417,53","2692117,33",0,0,0,"1089747423,93",...,"10812928,24",306080,"22442,79","4649015,53",0,"5010196,99","47589920,29",0,0,RREO_Estadual_12_6_2023_extraido.txt
3,Acre,1200,2024,"9260112468,41",1700903005,"802751,56",0,0,"10336361,22","1266787793,6",...,"66893078,47",0,"12953,78","5936684,18",0,"34522319,6","32216366,98",0,0,RREO_Estadual_12_6_2024_extraido.txt
4,Acrelândia,1200013,2021,"20359381,14","3646512,09","309259,11",0,0,N/A,"12203048,07",...,"150287,55",0,"255373,8","182270,94",0,0,0,0,0,RREO_Municipal_120001_6_2021_extraido.txt


## Conversão de tipos decimais

In [39]:
from decimal import Decimal, InvalidOperation

# Lista de colunas que devem ser convertidas
colunas_decimais = [
    'valor_receita_impostos', 'valor_repasse_fundeb', 'valor_minimo_mde',
    'valor_receita_vaaf', 'valor_receita_vaat', 'valor_receita_vaar',
    'valor_receita_total_fundeb', 'valor_total_despesa_impostos',
    'valor_receita_nao_aplicada', 'valor_receita_salario_educacao', 
    'valor_receita_pdde', 'valor_receita_pnae', 'valor_receita_pnate',
    'valor_receita_outras_fnde', 'valor_receita_convenios',
    'valor_receita_royalties', 'valor_receita_operacao_credito',
    'valor_receita_outras_outras'
]

# Função segura de conversão para Decimal
def converter_para_decimal(valor):
    try:
        return Decimal(str(valor).replace(',', '.'))
    except (InvalidOperation, TypeError, ValueError):
        return Decimal('0.00')

# Aplica a conversão para cada coluna
for coluna in colunas_decimais:
    df_panorama_financiamento[coluna] = df_panorama_financiamento[coluna].apply(converter_para_decimal)

df_panorama_financiamento[colunas_decimais].head(5)


,valor_receita_impostos,valor_repasse_fundeb,valor_minimo_mde,valor_receita_vaaf,valor_receita_vaat,valor_receita_vaar,valor_receita_total_fundeb,valor_total_despesa_impostos,valor_receita_nao_aplicada,valor_receita_salario_educacao,valor_receita_pdde,valor_receita_pnae,valor_receita_pnate,valor_receita_outras_fnde,valor_receita_convenios,valor_receita_royalties,valor_receita_operacao_credito,valor_receita_outras_outras
0,6205454230.86,1148021827.32,10164298.35,0,0,0.00,838181940.66,410900623.25,50307356.8,8908453.52,230100,13866212.8,2140819.51,1727377.07,388182.03,25262673.35,415570.13,4421500
1,7502282795.52,1393948222.07,7651815.7,0,0,0.00,1085913826.95,492393711.31,6604990.12,10243068.77,212280,0,0,0,0,51912871.88,0,0
2,7913861864.58,1449998417.53,2692117.33,0,0,0,1089747423.93,537924444.53,9345469.64,10812928.24,306080,22442.79,4649015.53,0,5010196.99,47589920.29,0,0
3,9260112468.41,1700903005,802751.56,0,0,10336361.22,1266787793.6,619560765.47,8584108.42,66893078.47,0,12953.78,5936684.18,0,34522319.6,32216366.98,0,0
4,20359381.14,3646512.09,309259.11,0,0,0.00,12203048.07,1969759.13,1354898.67,150287.55,0,255373.8,182270.94,0,0,0,0,0


## Criando colunas auxiliares

In [40]:
df_panorama_financiamento.columns

Index(['nome', 'cod_ibge', 'ano', 'valor_receita_impostos',
       'valor_repasse_fundeb', 'valor_minimo_mde', 'valor_receita_vaaf',
       'valor_receita_vaat', 'valor_receita_vaar',
       'valor_receita_total_fundeb', 'valor_total_despesa_impostos',
       'valor_receita_nao_aplicada', 'indicador_despesa_fundeb_profissionais',
       'valor_receita_salario_educacao', 'valor_receita_pdde',
       'valor_receita_pnae', 'valor_receita_pnate',
       'valor_receita_outras_fnde', 'valor_receita_convenios',
       'valor_receita_royalties', 'valor_receita_operacao_credito',
       'valor_receita_outras_outras', 'arquivo'],
      dtype='object')

### Coluna com valor total da receita adicional

In [41]:
from decimal import Decimal

# Lista de colunas que compõem a receita adicional
colunas_receitas_adicionais = [
    'valor_receita_salario_educacao',
    'valor_receita_pdde',
    'valor_receita_pnae',
    'valor_receita_pnate',
    'valor_receita_outras_fnde',
    'valor_receita_convenios',
    'valor_receita_royalties',
    'valor_receita_operacao_credito',
    'valor_receita_outras_outras'
]

# Soma linha a linha com precisão Decimal
df_panorama_financiamento['valor_total_receitas_adicionais'] = df_panorama_financiamento[colunas_receitas_adicionais].apply(
    lambda row: sum(row[col] for col in colunas_receitas_adicionais),
    axis=1
)

df_panorama_financiamento['valor_total_receitas_adicionais'].head(5)

0     57360888.41
1     62368220.65
2     68390583.84
3    139581403.01
4       587932.29
Name: valor_total_receitas_adicionais, dtype: object

### Coluna com o valor total dos recursos disponíveis para a educação

In [42]:
# Soma total receita Fundeb + receita impostos + receitas adicionais
from decimal import Decimal

# Substitui valores ausentes (caso existam) por Decimal('0.00') antes de somar
def soma_receita_educacao(row):
    return (
        (row.get('valor_receita_total_fundeb') or Decimal('0.00')) +
        (row.get('valor_total_despesa_impostos') or Decimal('0.00')) +
        (row.get('valor_total_receitas_adicionais') or Decimal('0.00'))
    )

# Cria a nova coluna com a soma
df_panorama_financiamento['valor_total_receita_educacao'] = df_panorama_financiamento.apply(
    soma_receita_educacao,
    axis=1
)

df_panorama_financiamento['valor_total_receita_educacao'].head(20)

0     1306443452.32
1     1640675758.91
2     1696062452.30
3     2025929962.08
4       14760739.49
5       22357605.65
6       22649618.43
7       24014752.44
8       10083995.42
9       11353339.80
10      13126679.09
11      16243319.35
12      23003335.05
13      30698316.59
14      35424543.84
15      42519107.68
16      12073041.49
17      14834157.98
18      16346063.49
19      21041431.56
Name: valor_total_receita_educacao, dtype: object

### Dropando colunas desnecessárias

In [43]:
df_panorama_financiamento.drop(columns=['arquivo'], inplace=True)

## Exportando o DF para CSV

In [44]:
df_panorama_financiamento.to_csv(
    Path("~/Projetos/Curica/curica_streamlit_censo/data/panorama_financiamento/df_panorama_financiamento.csv"),
    sep=";",
    encoding="utf-8",
    index=False
)